# Launch pytorch traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [2]:
import os
import glob,time
import subprocess as sp
import numpy as np


In [3]:
curr_dir=os.getcwd()
print(curr_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN


In [4]:
code_dirs={'128':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code',
          '512':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code' }

config={'128': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/config_128.yaml',
        '512': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/config_256.yaml'}

In [5]:
### Default dictionary
img_size=128
dict_pars={'epochs':20, 'batch_size':64, 'nodes':1,'seed':36723705, 'spec_loss': '','ip_fldr':'-',
           'job_name':'lambda0.1',
#            'ip_fldr':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201014_073327/',
           'spec_loss':'--specloss',
           'deterministic':'--deterministic',
           'save_steps_list':'11890 12580 13360 14070 32650 33240',
           'lr':0.0002,
           'lambda1':0.1, 
           'procs':8, 'time':'4:00:00','mode':'fresh'}

dict_pars['code_dir']=code_dirs[str(img_size)]
dict_pars['config_file']=config[str(img_size)]
print(dict_pars)

{'epochs': 20, 'batch_size': 64, 'nodes': 1, 'seed': 36723705, 'spec_loss': '--specloss', 'ip_fldr': '-', 'job_name': 'lambda0.1', 'deterministic': '--deterministic', 'save_steps_list': '11890 12580 13360 14070 32650 33240', 'lr': 0.0002, 'lambda1': 0.1, 'procs': 8, 'time': '4:00:00', 'mode': 'fresh', 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code', 'config_file': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/config_128.yaml'}


In [6]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node={procs}
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME
### Initial setup

conda activate v3
### module load pytorch-v1.4.0-gpu
##module load esslurm

code_dir={code_dir}
config_file={config_file}
python $code_dir/main.py --seed {seed} {spec_loss} {deterministic} -l {save_steps_list} --config $config_file --mode {mode} --ngpu {procs} --learn_rate {lr} --batchsize {batch_size} --epochs {epochs} -ld1 {lambda1} -ip {ip_fldr} -rs {job_name}

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [7]:
bash_strg

'#!/bin/bash\n#################\n#SBATCH --nodes=1\n#SBATCH --qos=regular\n#SBATCH --output=slurm-%x-%j.out\n#SBATCH --constraint=gpu\n#SBATCH --account=m3363\n#SBATCH --ntasks-per-node=8\n#SBATCH --cpus-per-task=8 \n#SBATCH --gpus-per-task=1\n#SBATCH --time=4:00:00\n#SBATCH --job-name=lambda0.1\n\necho "--start date" `date` `date +%s`\necho \'--hostname \' $HOSTNAME\n### Initial setup\n\nconda activate v3\n### module load pytorch-v1.4.0-gpu\n##module load esslurm\n\ncode_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code\nconfig_file=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/config_128.yaml\npython $code_dir/main.py --seed 36723705 --specloss --deterministic -l 11890 12580 13360 14070 32650 33240 --config $config_file --mode fresh --ngpu 8 --learn_rate 0.0002 --batchsize 64 --epochs 20 -ld1 0.1 -ip - -rs lambda0.1\n\necho "--end date" `date

In [8]:
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/'

In [9]:
fname='batch_train.sh'
filename=staging_loc+fname
with open (filename,'w') as f:
    f.write(bash_strg)

In [10]:
print(filename)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/batch_train.sh


In [11]:
### Move to staging locations in project space:
os.chdir(staging_loc)

## Submit job to cori GPU

In [12]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8 
#SBATCH --gpus-per-task=1
#SBATCH --time=4:00:00
#SBATCH --job-name=lambda0.1

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME
### Initial setup

conda activate v3
### module load pytorch-v1.4.0-gpu
##module load esslurm

code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code
config_file=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/config_128.yaml
python $code_dir/main.py --seed 36723705 --specloss --deterministic -l 11890 12580 13360 14070 32650 33240 --config $config_file --mode fresh --ngpu 8 --learn_rate 0.0002 --batchsize 64 --epochs 20 -ld1 0.1 -ip - -rs lambda0.1

echo "--end date" `date` `date +%s`


In [13]:
%%bash -s "$filename" ## Use python variable in bash
module load esslurm
sbatch $1

Submitted batch job 1290331
